## Script for generating temperature response to country-level emissions preturbation by way of FaIR
#### Mustafa Zahid, 05/27/22

In [1]:
#Importing main packages as well as packages to generate the distributions 
# from which we will pull the sets of parameters
from scipy.stats import lognorm, norm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Part I: Generating climatic parameters (following Ashwin et al., 2021)

In [2]:
# Writing the functions to generate distributions for our climate parameters (inputs in FaIR model)
def lognorm_from_percentiles(x1, p1, x2, p2):
    """ Return a log-normal distribuion X parametrized by:
    
            P(X < p1) = x1
            P(X < p2) = x2
    """
    x1 = np.log(x1)
    x2 = np.log(x2)
    p1ppf = norm.ppf(p1)
    p2ppf = norm.ppf(p2)
    
    scale = (x2 - x1) / (p2ppf - p1ppf)
    mean = ((x1 * p2ppf) - (x2 * p1ppf)) / (p2ppf - p1ppf)
    
    return np.random.lognormal(mean, scale, 1000)

In [3]:
# Writing the functions to generate distributions for our climate parameters (inputs in FaIR model)
def norm_from_percentiles(x1, p1, x2, p2):
    """ Return a normal distribuion X parametrized by:
    
            P(X < p1) = x1
            P(X < p2) = x2
    """
    p1ppf = norm.ppf(p1)
    p2ppf = norm.ppf(p2)

    location = ((x1 * p2ppf) - (x2 * p1ppf)) / (p2ppf - p1ppf)
    scale = (x2 - x1) / (p2ppf - p1ppf)

    return np.random.normal(location, scale, 1000)

In [4]:
# Now iterating over the parameters to pull from the distributions
tcr = []
rwf = []
tau = []
d2 = []
ecs = []
idi = []
for i in range(1,101):
        idi.append(i)
        tcr.append(np.random.choice(lognorm_from_percentiles(1, 0.17, 2.5, 0.83)))
        rwf.append(np.random.choice(norm_from_percentiles(0.45, 0.25, 0.75, 0.75)))
        tau.append(np.random.choice(np.random.normal(4.03, 1.79, 1000)))
        d2.append(np.random.choice(lognorm_from_percentiles(1.6, 0.05, 8.4, 0.95))) 

In [5]:
# Now generate a dataframe and add the pulled parameters together
df = pd.DataFrame()
df["rwfnum"] = rwf
df["taunum"] = tau
df["tcrnum"] = tcr
df["d2num"] = d2

In [6]:
# Following Ashwin et al., we restrict our parameters 
#tau
df = df[df["taunum"] > 0] 
df = df[df["taunum"] < (2*4.03)]

#rwf
df = df[df["rwfnum"] < 1]
df = df[df["rwfnum"] > np.percentile(df["rwfnum"],6)]

#ecs
df["ecs"] = df["tcrnum"] / df["rwfnum"]

In [7]:
#generate an ID for each of the sets of parameters
df["idnum"] = range(len(df))

### Part II: Running the FaIR model by iterating over emissions preturbation scenarios

In [8]:
# read in the needed libraries 
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from fair import *
#%matplotlib inline

In [9]:
# read the emissions country-year-level data
data = pd.read_csv('long_term_emissions_data_new.csv')  
data1 = pd.read_csv("country_lvl_emms_new.csv")

In [ ]:
#data = pd.read_csv('long_term_emissions_data_new_ssp126.csv')  

In [10]:
 # The following function returns an empty dataframe in the correct format for use in FaIR.
test_emissions = return_empty_emissions(df_to_copy=False, 
                                        start_year=0, 
                                        end_year=(350), 
                                        timestep=1, 
                                        scen_names=['Test'], 
                                        gases_in = ['carbon_dioxide'])

In [11]:
  # create an empty forcing dataframe compatible with test_emissions:
test_forcing = return_empty_forcing(test_emissions)

    # Note that the scenario names in the emissions and forcing dataframe must be identical for 
    #the model to run: FaIR assumes each emissions scenario corresponds directly to a single forcing scenario.
    
    # Generate a default parameter dataframe:
test_gas_parameters = get_gas_parameter_defaults()
test_thermal_parameters = get_thermal_parameter_defaults()

    # This is what the gas parameter dataframe looks like:
test_gas_parameters.head()
test_gas_parameters = test_gas_parameters.reindex(test_emissions.columns.levels[1],
                                                  axis=1,
                                                  level=1)

In [12]:
list_of_isos = pd.unique(data1["ISO3"])

In [ ]:
master_array = np.array_split(list_of_isos, 2) 

In [ ]:
list_dfs = []
for m in range(1,len(df)): 
    for i in master_array[0]:
        # shut off emissions from country i
        data_iso = data1[data1["ISO3"] == i]
        data_iso = data_iso[["Year", "Total"]]
        data_iso.rename(columns = {'Total':'Total_iso'}, inplace = True)
        data_new = pd.merge(data, data_iso, on = "Year")
        data_new["Total"] = data_new["Total"] - data_new["Total_iso"]
        data_new = data_new[["Year", "Total"]]
        data_new.rename(columns = {'Total':'carbon_dioxide'}, inplace = True) #change name to match emissions data
        data_new.rename(columns = {'Year':'year'}, inplace = True) #change name to match emissions data
        data_new["year"] = pd.to_numeric(data_new["year"]) 
        # since the carbon emissions are on MtCO2, we need to convert to GtC
        data_new['carbon_dioxide'] = (data_new['carbon_dioxide'] / 1000) / 3.67 
        data_new = data_new[data_new["year"] >= 1980].reset_index() # only keep 1980 and forward
        data_new['year'] = data_new['year'] - (min(data_new['year']) - 1) 
        #feed updated emissions to test_emissions
        test_emissions[("Test", "carbon_dioxide")] = data_new['carbon_dioxide']
        test_gas_parameters[("default", "carbon_dioxide")][7] = list(df1["taunum"])[0]
        
        # therefore use central estimates for d1-3, q1; and set q2, q3 to return "correct" TCR/ECS values
        df1 = df[df['idnum'] == m] 
        d1 = 0.903
        d2 = list(df1["d2num"])[0]
        d3 = 355
        q1 = 0.180
        rwf = list(df1["rwfnum"])[0]

        TCR = list(df1["tcrnum"])[0]
        ECS = TCR / rwf
        F_2x = 3.759

        v1 = (1-(d1/69.66) * (1-np.exp(-69.66/d1)) )
        v2 = (1-(d2/69.66) * (1-np.exp(-69.66/d2)) )
        v3 = (1-(d3/69.66) * (1-np.exp(-69.66/d3)) )

        q3 = (((TCR/F_2x) - q1*(v1-v2) - (ECS/F_2x)*v2) / (v3-v2))
        q2 = (ECS/F_2x - q1 -  q3)

        test_thermal_parameters["default",2][1] = q2
        test_thermal_parameters["default",3][1] = q3

            #test_thermal_parameters = pd.DataFrame([[d1,d2,d3],[q1,q2,q3]],
             #                                      index=['d','q'],
              #                                     columns=pd.MultiIndex.from_product([['default'],
                      #                                                                 [1,2,3]]))
        pulse_run = run_FaIR(emissions_in=test_emissions,
                             forcing_in=test_forcing,
                             gas_parameters=test_gas_parameters,
                             thermal_parameters=test_thermal_parameters)
        temp_response = pulse_run['T']
        #(temp_response.temp_response).apply(lambda x: float(x))   
            #temp_response["temp_response"] = float(temp_response["temp_response"]) 
            #temp_response.to_csv('temp_response/tempresponse' + "_" + str(k) + "_" + str(iso) + '.csv') 
        temp_response["loop"] = str(m) + "_loop_" + i   
        list_dfs.append(temp_response)

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_fut0.csv")

In [ ]:
list_dfs = []
for m in range(1,len(df)): 
  #  for i in master_array[9]:
        # shut off emissions from country i
#        data_iso = data1[data1["ISO3"] == i]
 #       data_iso = data_iso[["Year", "Total"]]
  #      data_iso.rename(columns = {'Total':'Total_iso'}, inplace = True)
   #     data_new = pd.merge(data, data_iso, on = "Year")
    #    data_new["Total"] = data_new["Total"] - data_new["Total_iso"]
    data_new = data
    data_new = data_new[["Year", "Total"]]
    data_new.rename(columns = {'Total':'carbon_dioxide'}, inplace = True) #change name to match emissions data
    data_new.rename(columns = {'Year':'year'}, inplace = True) #change name to match emissions data
    data_new["year"] = pd.to_numeric(data_new["year"]) 
    # since the carbon emissions are on MtCO2, we need to convert to GtC
    data_new['carbon_dioxide'] = (data_new['carbon_dioxide'] / 1000) / 3.67 
    #data_new = data_new[data_new["year"] >= 1980].reset_index() # only keep 1980 and forward
    data_new['year'] = data_new['year'] - (min(data_new['year']) - 1) 
    #feed updated emissions to test_emissions
    test_emissions[("Test", "carbon_dioxide")] = data_new['carbon_dioxide']
    test_gas_parameters[("default", "carbon_dioxide")][6] = list(df1["taunum"])[0]
        
        # therefore use central estimates for d1-3, q1; and set q2, q3 to return "correct" TCR/ECS values
    df1 = df[df['idnum'] == m] 
    d1 = 0.903
    d2 = list(df1["d2num"])[0]
    d3 = 355
    q1 = 0.180
    rwf = list(df1["rwfnum"])[0]

    TCR = list(df1["tcrnum"])[0]
    ECS = TCR / rwf
    F_2x = 3.759

    v1 = (1-(d1/69.66) * (1-np.exp(-69.66/d1)) )
    v2 = (1-(d2/69.66) * (1-np.exp(-69.66/d2)) )
    v3 = (1-(d3/69.66) * (1-np.exp(-69.66/d3)) )

    q3 = (((TCR/F_2x) - q1*(v1-v2) - (ECS/F_2x)*v2) / (v3-v2))
    q2 = (ECS/F_2x - q1 -  q3)

    test_thermal_parameters["default",2][1] = q2
    test_thermal_parameters["default",3][1] = q3

            #test_thermal_parameters = pd.DataFrame([[d1,d2,d3],[q1,q2,q3]],
             #                                      index=['d','q'],
              #                                     columns=pd.MultiIndex.from_product([['default'],
                      #                                                                 [1,2,3]]))
    pulse_run = run_FaIR(emissions_in=test_emissions,
                         forcing_in=test_forcing,
                         gas_parameters=test_gas_parameters,
                         thermal_parameters=test_thermal_parameters)
    temp_response = pulse_run['T']
        #(temp_response.temp_response).apply(lambda x: float(x))   
            #temp_response["temp_response"] = float(temp_response["temp_response"]) 
            #temp_response.to_csv('temp_response/tempresponse' + "_" + str(k) + "_" + str(iso) + '.csv') 
    temp_response["loop"] = str(m) + "_loop_all" 
    list_dfs.append(temp_response)

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_1750_2100.csv")

### end of script